## 8.2 レーダーチャートとポーラーチャート

In [1]:
import json
import pandas as pd

from plotly.subplots import make_subplots
from plotly import data as pdata
from plotly import graph_objects as go
from plotly.graph_objs.layout import Template

# WindデータセットのDataFrameを読み込み
df = pdata.wind()

df

,direction,strength,frequency
0,N,0-1,0.5
1,NNE,0-1,0.6
2,NE,0-1,0.5
3,ENE,0-1,0.4
4,E,0-1,0.4
...,...,...,...
123,WSW,6+,0.1
124,W,6+,0.9
125,WNW,6+,2.2
126,NW,6+,1.5


In [2]:
# 変数「strength」で一意な値を確認
df['strength'].unique()

array(['0-1', '1-2', '2-3', '3-4', '4-4', '4-5', '5-6', '6+'],
      dtype=object)

In [3]:
# 変数「strength」の値でグループ化
gb = df.groupby('strength')

# たとえば '0-1' のDataFrame
df_subset = gb.get_group('0-1')

df_subset

,direction,strength,frequency
0,N,0-1,0.5
1,NNE,0-1,0.6
2,NE,0-1,0.5
3,ENE,0-1,0.4
4,E,0-1,0.4
5,ESE,0-1,0.3
6,SE,0-1,0.4
7,SSE,0-1,0.4
8,S,0-1,0.6
9,SSW,0-1,0.4


In [4]:
groups = ['0-1', '2-3', '4-5', '6+']    # 使用するグループ

# Traceのlistを作成
traces = []
for group in groups:
    df_subset = gb.get_group(group)
    
    trace = go.Scatterpolar(
        theta=df_subset['direction'],   # 角度に使用する変数
        r=df_subset['frequency'],       # 動経に使用する変数
        mode='lines',                   # グラフモード（線あり）
        fill='none',                    # 塗りつぶし（なし）
        name=group
    )   # 方位別の風向き頻度を示したレーダーチャート
    traces.append(trace)

# 独自テンプレートを読み込み
with open('custom_white.json') as f:
    custom_white_dict = json.load(f)
    template = Template(custom_white_dict)

# Layoutを作成
layout = layout=go.Layout(
    template=template,
    title='Wind frequency',
)

# Figureを作成
figure = go.Figure(traces, layout)

figure

In [5]:
# Traceのlistを作成
traces = []
for group in groups:
    # 末尾に冒頭を連結したDataFrameを作成する
    df_subset = pd.concat([gb.get_group(group), gb.get_group(group).iloc[[0]]], axis=0)

    trace = go.Scatterpolar(
        theta=df_subset['direction'],
        r=df_subset['frequency'],
        mode='lines',
        fill='none',
        name=group
    )
    traces.append(trace)

# Layoutに使用する極座標の設定
polar_dict = {
    'angularaxis': {
        'visible': True,
        'rotation': 90,             # 回転（90度回転）
        'direction': 'clockwise',   # 方向（時計回り）
    },  # 角度の軸
    'radialaxis': {
        'range': [0., 2.4], # 範囲
    }   # 動経の軸
}

# Layoutを作成
layout = go.Layout(
    template=template,
    title='Wind frequency',
    polar=polar_dict
)

# Figureを作成
figure = go.Figure(traces, layout)

figure

In [6]:
# Figureを作成
figure = make_subplots(
    rows=1,
    cols=4,
    horizontal_spacing=0.1,
    subplot_titles=[f'strength: {group}' for group in groups],
    specs=[[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
)

# TraceをFigureに追加
for i, group in enumerate(groups, start=1):
    df_subset = gb.get_group(group)

    trace = go.Scatterpolar(
        theta=df_subset['direction'],
        r=df_subset['frequency'],
        mode='none',        # グラフモード（線なし）
        fill='toself',      # 塗りつぶし（あり）
        name=group
    )
    figure.add_trace(
        trace,
        row=1,
        col=i
    )

# Layoutを更新
figure.update_layout(
    go.Layout(
        template=template,
        title='Wind frequency',
        polar=polar_dict,
        polar2=polar_dict,
        polar3=polar_dict,
        polar4=polar_dict
    )
)

figure

In [7]:
# Figureを作成
figure = make_subplots(
    rows=1,
    cols=4,
    horizontal_spacing=0.1,
    subplot_titles=[f'strength: {group}' for group in groups],
    specs=[[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
)

# TraceをFigureに追加
for i, group in enumerate(groups, start=1):
    df_subset = gb.get_group(group)

    trace = go.Barpolar(
        theta=df_subset['direction'],
        r=df_subset['frequency'],
        name=group
    )   # 方位別の風向き頻度を示したポーラーチャート
    figure.add_trace(
        trace,
        row=1,
        col=i
    )

# Layoutを更新
figure.update_layout(
    go.Layout(
        template=template,
        title='Wind frequency',
        polar=polar_dict,   # 各サブプロットに極座標設定を適用
        polar2=polar_dict,
        polar3=polar_dict,
        polar4=polar_dict
    )
)

figure